## Reverse Neutralization Example to generate a dataset for fine-tuning

In [1]:
# pip install --upgrade --quiet  openai python-dotenv tqdm

In [1]:
import json
import os
import time
from dotenv import load_dotenv
from openai import OpenAI
from tqdm.auto import tqdm

load_dotenv("../keys.env")

assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please sign up for access to the OpenAI API and provide access token in keys.env file"

In [2]:
# Initialize the OpenAI client
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]
)

In [4]:
# read the collection of emails
emails = []
with open("emails.jsonl", "r") as f:
    for line in f:
        emails.append(json.loads(line))


In [5]:
# Neutralize the emails

prompt = """
Neutralize the tone and style from the following email to make it professional and suitable for communication between executives who may not know each other very well.

{email}
"""

neutralized_emails = []

for email in tqdm(emails):
    prompt_with_email = prompt.format(email=email)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_with_email}]
    )

    neutralized_emails.append(response.choices[0].message.content)


  0%|          | 0/198 [00:00<?, ?it/s]

### Generate the dataset for fine-tuning

In [6]:
dataset = []

for email, neutralized_email in zip(emails, neutralized_emails):
    dataset.append({
        "messages": [
            {"role": "system", "content": "You are a helpful assistant converting the neutralized email into personalized email."},
            {"role": "user", "content": neutralized_email},
            {"role": "assistant", "content": email}
        ]
    })

# write out the dataset to a jsonl file
with open("dataset.jsonl", "w") as f:
    for item in dataset:
        f.write(json.dumps(item) + "\n")


In [7]:
# show comparison the neutralized email and the personalized email, limit example to 1

print(f"Neutralized email: {neutralized_emails[0]}")
print("-"*100)
print(f"Personalized email: {emails[0]}")


Neutralized email: Subject: Summary of Marketing Team Meeting - {date}

Body: Good day Marketing team,

I trust this message finds you well. I am reaching out to provide a recap of our recent team meeting held on {date} at {time} in {location}.

During our discussion on the Q2 roadmap, we addressed the following key points:
- Established alignment on priorities for the upcoming quarter
- Explored and deliberated on challenges alongside potential resolutions
- Revised the timeline and outlined deliverables

Moving forward, our next actions will include:
- Scheduling a follow-up meeting for next week
- Distributing updated documentation by this Friday

Should you have any inquiries or if I inadvertently overlooked any crucial details, please do not hesitate to reach out.

Warm regards,
Evelyn
----------------------------------------------------------------------------------------------------
Personalized email: Subject: Marketing Team Meeting Summary - {date}

Body: Dear Marketing team,


### Observation:

The model is able to handle the task, but it fails on the correct placeholder tags or isn't hitting the correct tone you prefer. Let's fix those issues by fine-tuning a model.


In [8]:
# Upload the training file
training_file = client.files.create(
    file=open("dataset.jsonl", "rb"),
    purpose="fine-tune"
)

In [9]:

# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo"  # Base model to fine-tune
)


In [10]:
# Continuously check the status of the fine-tuning job
while True:
    job_status = client.fine_tuning.jobs.retrieve(job.id)
    print(f"Job status: {job_status.status}")

    if job_status.status in ['succeeded', 'failed']:
        break

    print("Waiting 120 seconds...")
    time.sleep(120)

if job_status.status == 'succeeded':
    print(f"Fine-tuning complete! You can now use model: {job_status.fine_tuned_model}")
else:
    print("Fine-tuning failed. Check the job status for more information.")

# Once the job is complete, you can use the fine-tuned model
# The fine-tuned model ID will be available in job_status.fine_tuned_model

Job status: validating_files
Waiting 120 seconds...
Job status: queued
Waiting 120 seconds...
Job status: queued
Waiting 120 seconds...
Job status: queued
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: running
Waiting 120 seconds...
Job status: succeeded
Fine-tuning complete! You can now use model: ft:gpt-3.5-turbo-0125:digits::B7zqBVSB


In [11]:
# Now we can use the fine-tuned model to generate the email with neutralized_emails[0]
# Generally it is ad practise to test the model with a sample input from the training data,
# but we want to check the output of the fine-tuned model.

# Test the fine-tuned model with a sample input neutralized_emails[0]
completion = client.chat.completions.create(
    model=job_status.fine_tuned_model,  # Use the fine-tuned model
    messages=[
        {"role": "system", "content": "You are a helpful assistant converting the neutralized email into personalized email."},
        {"role": "user", "content": neutralized_emails[0]},]
)

# Print the generated response
print("Generated personalized email:")
print(completion.choices[0].message.content)

Generated personalized email:
Subject: Marketing Team Meeting Summary - {date}

Body: Dear Marketing team,

I hope this email finds you well. I'm writing to summarize our team meeting that took place on {date} at {time} in {location}.

During our discussion about Q2 roadmap, we covered several key points:
- Aligned on priorities for the next quarter
- Discussed challenges and potential solutions
- Updated timeline and deliverables

Next steps:
- Schedule follow-up meeting next week
- Share updated documentation by Friday

Please let me know if you have any questions or if I missed anything important.

Best regards,
Evelyn


In [12]:
# Now we can use the fine-tuned model to generate the email

test_email = """
Subject: Request for Project Timeline Update

Body: Hi Sam,

I am writing to request an update on the project timeline. Please provide the update by the end of the day, as it is important for our upcoming steps.

Thank you.

Best,
Alex
"""

# Test the fine-tuned model with a sample input
completion = client.chat.completions.create(
    model=job_status.fine_tuned_model,  # Use the fine-tuned model
    messages=[
        {"role": "system", "content": "You are a helpful assistant converting the neutralized email into personalized email."},
        {"role": "user", "content": test_email}
    ]
)

# Print the generated response
print("Generated personalized email:")
print(completion.choices[0].message.content)


Generated personalized email:
Subject: Request for project timeline update

Body: Hi Sam,

I hope you're doing well. I'm reaching out because I need project timeline.

This is needed for our upcoming deliverables.

Could you please help me with this? Let me know if you need any additional information.

Thank you in advance for your help.

Best,
Alex


## Inference Example

In [3]:
prompt = """
Write a short email to Gretl inviting her to give a presentation on the marketing campaign around the 2026 FIFA World Cup.
"""

response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant writing letters suitable for communication between executives."},
            {"role": "user", "content": prompt}]
    )

neutral_email = response.choices[0].message.content

print(neutral_email)

Subject: Invitation to Present on Marketing Campaign for 2026 FIFA World Cup

Dear Gretl,

I hope this message finds you well. I am writing to officially invite you to give a presentation on the marketing campaign surrounding the 2026 FIFA World Cup. Your expertise and insights would be invaluable to our team, and we are eager to hear your thoughts on this exciting project.

We believe that your unique perspective and experience will bring a fresh and innovative approach to our marketing strategies for this upcoming event. Your presentation will provide valuable insights that will help guide our team in creating a successful campaign.

Please let me know at your earliest convenience if you are available and willing to present. We are looking forward to hearing from you and are excited about the opportunity to collaborate on this important project.

Thank you in advance for considering our invitation. We appreciate your time and expertise.

Warm regards,

[Your Name]
[Your Title]
[Compa

In [ ]:
completion = client.chat.completions.create(
    model=job_status.fine_tuned_model,  # Use the fine-tuned model
    messages=[
        {"role": "system", "content": "You are a helpful assistant converting the neutralized email into personalized email."},
        {"role": "user", "content": neutral_email}
    ]
)

# Print the generated response
print("Generated personalized email:")
print(completion.choices[0].message.content)

Generated personalized email:
Subject: Invitation to present about marketing campaign for 2026 FIFA World Cup

Body: Dear Gretl,

I hope you're doing well. I'm reaching out because I'd like to invite you to present about marketing campaign for 2026 FIFA World Cup. I think your insights would be incredibly valuable in shaping our team's priorities and I'd love to discuss this opportunity with you in more detail.

Please let me know if you're interested in discussing this further, or if you have any questions about what the presentation would entail.
